In [1]:
import numpy as np

### Joint distribution of two discrete random variables
The probabilities are organized in a 2D array, where the columns correspond to values of $x$ and the rows correspond to values of $y$

In [2]:
#### We start with positive weights that don't sum to 1
P=np.array([[2.,2,4],[1,1,2]])
P2=P
print(P)

[[ 2.  2.  4.]
 [ 1.  1.  2.]]


In [3]:
P[0,0]=0

In [4]:
print(P2)

[[ 0.  2.  4.]
 [ 1.  1.  2.]]


In [8]:
P=np.array([[2.,2,4],[1,1,2]])

In [9]:
P2=np.copy(P)
P[0,0]=0
print(P2)

[[ 2.  2.  4.]
 [ 1.  1.  2.]]


In [18]:
P = np.copy(P2)

In [19]:
# Nrmalize the weights using basic Python

#Compute the sum
s=0
for i in range(P.shape[0]):
    for j in range(P.shape[1]):
        s+=P[i,j]
print('the sum is ',s)
#divide by the sum
for i in range(P.shape[0]):
    for j in range(P.shape[1]):
        P[i,j] /= s
P

the sum is  1.0


array([[ 0.16666667,  0.16666667,  0.33333333],
       [ 0.08333333,  0.08333333,  0.16666667]])

In [20]:
# Using Numpy we can write it in a much shorter way
P2/=np.sum(P2)
P2

array([[ 0.16666667,  0.16666667,  0.33333333],
       [ 0.08333333,  0.08333333,  0.16666667]])

In [21]:
# The values that the random variables x and y take
x=np.array([1,2,6])
y=np.array([-1,1])

In [23]:
P.shape

(2, 3)

#### Computing Marginals
The marginal distributions are the probabilities associated with each random variable alone.

In [25]:
# The pure python way
Px=[0.]*P.shape[1]
Py=[0.]*P.shape[0]

for i in range(len(Px)):
    for j in range(len(Py)):
        Px[i]+=P[j,i]
        Py[j]+=P[j,i]
Px,Py

([0.25, 0.25, 0.5], [0.66666666666666663, 0.33333333333333331])

In [27]:
#the numpy way:
Px=np.sum(P,axis=0)
Py=np.sum(P,axis=1)
Px,Py

(array([ 0.25,  0.25,  0.5 ]), array([ 0.66666667,  0.33333333]))

### Testing independence

If two random variables $X$ and $Y$ are independent, then the outer product of the marginals should equal P.

In [32]:
# The pure python way
for i in range(Px.size):
    for j in range(Py.size):
        if Px[i]*Py[j] != P[j,i]:
            print("Px[%d]*Py[%d] != P[%d,%d] ::::: %5.3f*%5.3f = %5.3f != %5.3f"%(i,j,j,i,Px[i],Py[j],Px[i]*Py[j],P[j,i]))


In [33]:
# The numpy way
np.outer(Px,Py).T - P

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

### Computing the covariance

### Calculating the mean and standard deviation
To calculate the mean of $X$ and $Y$ under this distribtion in python, we need to iterate through the values of $x$ and $y$ and plug them into the formuls $E[X] = \sum_x P(X=x)x$. Similarly for standard deviation.


In [34]:
from math import sqrt
#The python way
Ex = 0
for i in range(3):
    Ex+=Px[i]*x[i]
Ey = 0
for i in range(2):
    Ey+=Py[i]*y[i]

varx = 0
for i in range(3):
    varx+=Px[i]*(x[i] - Ex)**2
stdx = sqrt(varx)

vary = 0
for i in range(2):
    vary+=Py[i]*(y[i] - Ey)**2
stdy = sqrt(vary)

Ex,Ey,stdx,stdy

(3.75, -0.33333333333333331, 2.277608394786075, 0.9428090415820634)

In [35]:
# In numpy you can use np.dot(A,B) which calculates the pairwise product of elements in A and B
# and sums them up
Ex=np.dot(Px,x)
Ey=np.dot(Py,y)
Ex2=np.dot(Px,x**2)
Ey2=np.dot(Py,y**2)
stdx=sqrt(Ex2-Ex**2)
stdy=sqrt(Ey2-Ey**2)
print('Ex=%f,Ey=%f,stdx=%f,stdy=%f'%(Ex,Ey,stdx,stdy))

Ex=3.750000,Ey=-0.333333,stdx=2.277608,stdy=0.942809


#### Subtract the means

In [36]:
nx=x-Ex
nx

array([-2.75, -1.75,  2.25])

In [37]:
ny=y-Ey
ny

array([-0.66666667,  1.33333333])

### Calculate the covariance


In [38]:
# in python
s=0
for i in range(len(x)):
    for j in range(len(y)):
        s+=P[j,i]*nx[i]*ny[j]
print('the covariance is',s) #our expected values are now 0 so nothing to subtract

the covariance is 1.11022302463e-16


In [39]:
# numpy

print('the covariance is', np.dot(P.flatten(), np.outer(ny,nx).flatten()))

the covariance is 0.0


### and the correlation


In [40]:
s/(stdx*stdy),stdx,stdy

(5.1702011052845407e-17, 2.277608394786075, 0.9428090415820634)

In [41]:
import numpy as np
from math import sqrt

def ComputeStatistics(P,x,y):
    P/=np.sum(P) # normalize the disttribution
    Px=np.sum(P,axis=0) # compute marginals
    Py=np.sum(P,axis=1)
    Ex=np.dot(Px,x)
    Ey=np.dot(Py,y)
    Ex2=np.dot(Px,x**2)
    Ey2=np.dot(Py,y**2)
    stdx=sqrt(Ex2-Ex**2)
    stdy=sqrt(Ey2-Ey**2)
    
    nx=x-Ex
    ny=y-Ey
    
    cov=np.dot(P.flatten(), np.outer(ny,nx).flatten())
    corr=cov/(stdx*stdy)
    return {
        'P':P,
        'x':x,
        'y':y,
        'Px':Px,
        'Py':Py,
        'Ex':Ex,
        'Ey':Ey,
        'stdx':stdx,
        'stdy':stdy,
        'cov':cov,
        'corr':corr
    }


In [42]:
x=np.arange(1.,2.,0.2)
y=np.arange(0.,1.,0.2)
P=np.eye(5)
print(x,y)
P

[ 1.   1.2  1.4  1.6  1.8] [ 0.   0.2  0.4  0.6  0.8]


array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [43]:
A=ComputeStatistics(P,x,y)
print(A['cov'],A['corr'])

0.08 1.0


## Empirical statistics

If we now draw samples from these distributions, we can see that the emperical statistics, the population mean, population standard deviation and population covariance approach the original values of mean, standard deviation and covariance.

In [44]:
P=np.array([[1.,1,1],[1,1,2],[2,1,1]])
x=np.array([-1,0,1]); y=np.array([-1,0,1])
A=ComputeStatistics(P,x,y)
Px=A['Px']
Px,A['Ex'],A['cov']

(array([ 0.36363636,  0.27272727,  0.36363636]), 0.0, -0.090909090909090912)

In [45]:
Px

array([ 0.36363636,  0.27272727,  0.36363636])

In [46]:
np.random.choice(x,100,True, Px)

array([ 1,  1,  1,  0,  1,  0, -1, -1, -1, -1, -1,  1,  0, -1, -1,  1,  1,
       -1, -1,  1, -1,  0,  0, -1,  1,  1, -1,  1,  1, -1,  1,  1,  1, -1,
        1,  0,  0, -1, -1,  1,  0,  0, -1,  1, -1,  0,  0,  1,  1, -1,  1,
        1,  1, -1,  1,  1, -1,  0,  1,  0, -1, -1, -1, -1, -1,  1, -1,  0,
       -1, -1,  1,  1, -1, -1,  0,  1,  0,  1,  1, -1,  0,  1, -1,  0, -1,
        0,  1,  0,  0, -1, -1, -1,  0,  1, -1,  1,  0, -1, -1,  0])

In [47]:
numsamples = [2,10,100,100000]

for num in numsamples: 
    print("Sample mean after drawing {num} samples = {s}".format(
        num = num,
        s = np.mean(np.random.choice(x, num, True, Px))
    ))

Sample mean after drawing 2 samples = -0.5
Sample mean after drawing 10 samples = 0.2
Sample mean after drawing 100 samples = -0.07
Sample mean after drawing 100000 samples = 0.00261


In [48]:
#To calculate the covariance, we will generate samples (x,y) form the joint distribution P
#possible samples
xy =  np.array([(i,j) for i in x for j in y])
I=np.random.choice(xy.shape[0], num, True, P.T.flatten())
xy[I][:20]

array([[ 1,  0],
       [ 1,  1],
       [ 0, -1],
       [ 1,  0],
       [-1, -1],
       [ 0, -1],
       [ 0,  1],
       [ 1,  0],
       [ 0,  1],
       [-1,  1],
       [-1,  1],
       [-1, -1],
       [ 1,  0],
       [ 0,  1],
       [-1,  1],
       [ 1,  0],
       [ 0,  1],
       [ 1, -1],
       [ 1,  0],
       [-1, -1]])

In [49]:
for num in numsamples:
    samples = np.random.choice(xy.shape[0], num, True, P.T.flatten()), #choose rows
    print("Population covariance after drawing {num} samples = {s}".format(
        num = num,
        s = np.cov(
            xy[samples][:,0],
            xy[samples][:,1]
        )[0,1]
    ))

Population covariance after drawing 2 samples = 0.0
Population covariance after drawing 10 samples = 0.02222222222222222
Population covariance after drawing 100 samples = -0.20424242424242425
Population covariance after drawing 100000 samples = -0.09311053000529987
